In [1]:
# Add dependencies
using Pkg
Pkg.add(["CSV", "Optimisers", "Lux", "LuxCUDA", "DataFrames", "Zygote", "ComponentArrays", "Plots"])

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [2]:
# Load dependencies
using CSV, Lux, LuxCUDA, Optimisers, Random, DataFrames, Zygote, ComponentArrays

In [ ]:


# Define enums
@enum PoisonClass Poisonous Edible

@enum Amount begin
    None
    One
    Two
end

@enum CapShape begin
    Bell
    Conical
    Convex
    Flat
    Knobbed
    Sunken
end

@enum Surface begin
    Fibrous
    Grooves
    Scaly
    Silky
    Smooth
end



@enum Bruised begin
    Bruises
    NoBruises
end

@enum Odor begin
    Almond
    Anise
    Creosote
    Fishy
    Foul
    Musty
    Pungent
    Spicy
end

@enum Huh begin
    Something
end

@enum GillAttachment begin
    Attached
    Descending
    Free
    Notched
end

@enum Spacing begin
    Close
    Crowded
    Distant
end

@enum GillSize begin
    Broad
    Narrow
end

@enum MColors begin
    Black
    Brown
    Buff
    Chocolate
    Cinnamon
    Gray
    Green
    Orange
    Pink
    Purple
    Red
    White
    Yellow
end

@enum StalkShape begin
    Enlarging
    Tapering
end

@enum StalkRoot begin
    Bulbous
    Club
    Cup
    Equal
    Rooted
end


@enum VeilType begin
    Partial
    Universal
end

@enum RingType begin
    Cobwebby 
    Evanescent
    Flaring
    Large 
    Pendant
    Sheathing 
    Zone
end




@enum Population begin
    Abundant
    Clustered
    Numerous
    Scattered
    Several
    Solitary
end




@enum Habitat begin
    Grasses
    Leaves 
    Meadows
    Paths 
    Urban
    Waste
    Woods
end





In [9]:



# Define data
data = DataFrame(CSV.File(raw"mushrooms.csv"))

macro Enumify(data, column, args...)
    # Generate a list of if expressions
    expr = :x
    for i in 1:2:length(args)
        symbol = args[i]
        value = args[i+1]
        expr = Expr(:if, :(x == $value), :($symbol), expr)
    end

    quote
        $(esc(data))[!, $(esc(column))] = map(x -> $expr, $(esc(data))[!, $(esc(column))])
    end

end

# Label data
@Enumify data :class Poisonous "p" Edible "e"
@Enumify data "cap-shape" Bell "b" Conical "c" Convex "x" Flat "f" Knobbed "k" Sunken "s"
@Enumify data "cap-surface" Fibrous "f" Grooves "g" Scaly "y" Smooth "s"
@Enumify data "cap-color" Brown "n" Buff "b" Cinnamon "c" Gray "g" Green "r" Pink "p" Purple "u" Red "e" White "w" Yellow "y"
@Enumify data :bruises Bruises "t" NoBruises "f"
@Enumify data :odor Almond "a" Anise "l" Creosote "c" Fishy "y" Foul "f" Musty "m" None "n" Pungent "p" Spicy "s"
@Enumify data "gill-attachment" Attached "a" Descending "d" Free "f" Notched "n"
@Enumify data "gill-spacing" Close "c" Crowded "w" Distant "d"

# gill-size: broad=b,narrow=n
@Enumify data "gill-size" Broad "b" Narrow "n"




# gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
@Enumify data "gill-color" Black "k" Brown "n" Buff "b" Chocolate "h" Gray "g" Green "r" Orange "o" Pink "p" Purple "u" Red "e" White "w" Yellow "y"

# stalk-shape: enlarging=e,tapering=t
@Enumify data "stalk-shape" Enlarging "e" Tapering "t"

# stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r,missing=?
@Enumify data "stalk-root" Bulbous "b" Club "c" Cup "u" Equal "e" Rhizomorphs "z" Rooted "r" Missing "?"

# stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
@Enumify data "stalk-surface-above-ring" Fibrous "f" Scaly "y" Silky "k" Smooth "s"

# stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
@Enumify data "stalk-surface-below-ring"  Fibrous "f" Scaly "y"  Silky "k"  Smooth "s"


# stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
@Enumify data "stalk-color-above-ring"   Brown "n"  Buff "b" Cinnamon "c"  Gray "g"  Orange "o"  Pink "p"  Red "e"  White "w"  Yellow "y"
    

# stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
@Enumify data "stalk-color-below-ring"  Brown "n"  Buff "b"  Cinnamon "c" Gray "g"  Orange "o"  Pink "p"   Red "e"  White "w"  Yellow "y"


# veil-type: partial=p,universal=u
@Enumify data "veil-type"  Partial "p"  Universal "u"


# veil-color: brown=n,orange=o,white=w,yellow=y
@Enumify data "veil-color"  Brown "n"  Orange "o" White "w"  Yellow "y"

    

# ring-number: none=n,one=o,two=t
@Enumify data "ring-number"   None "n"   1 "o"   2 "t"


# ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z
@Enumify data "ring-type"   Cobwebby "c"  Evanescent "e"    Flaring "f"   Large "l"   None "n"  Pendant "p"   Sheathing "s"  Zone "z"


# spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y
@Enumify data "spore-print-color"  Black "k"   Brown "n"   Buff "b"   Chocolate "h"   Green "r"   Orange "o"   Purple "u"   White "w"   Yellow "y"


# population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y
@Enumify data "population"   Abundant "a"   Clustered "c"   Numerous "n"   Scattered "s"   Several "v"   Solitary "y"


# habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d
@Enumify data "habitat"   Grasses "g"   Leaves "l"    Meadows "m"   Paths "p"   Urban "u"   Waste "w"   Woods "d"




first(data, 5)

Row,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
,PoisonCl…,CapShape,Surface,MColors,Bruised,Enum…,GillAtta…,Spacing,GillSize,MColors,StalkSha…,Any,Surface,Surface,MColors,MColors,VeilType,MColors,Any,Enum…,MColors,Populati…,Habitat
1,Poisonous,Convex,Smooth,Brown,Bruises,Pungent,Free,Close,Narrow,Black,Enlarging,Equal,Smooth,Smooth,White,White,Partial,White,1,Pendant,Black,Scattered,Urban
2,Edible,Convex,Smooth,Yellow,Bruises,Almond,Free,Close,Broad,Black,Enlarging,Club,Smooth,Smooth,White,White,Partial,White,1,Pendant,Brown,Numerous,Grasses
3,Edible,Bell,Smooth,White,Bruises,Anise,Free,Close,Broad,Brown,Enlarging,Club,Smooth,Smooth,White,White,Partial,White,1,Pendant,Brown,Numerous,Meadows
4,Poisonous,Convex,Scaly,White,Bruises,Pungent,Free,Close,Narrow,Brown,Enlarging,Equal,Smooth,Smooth,White,White,Partial,White,1,Pendant,Black,Scattered,Urban
5,Edible,Convex,Smooth,Gray,NoBruises,None,Free,Crowded,Broad,Black,Tapering,Equal,Smooth,Smooth,White,White,Partial,White,1,Evanescent,Brown,Abundant,Grasses


In [ ]:
function split_df_percent(df, pct1, pct2; shuffled=true)
    if shuffled
        processed_df = df[shuffle(axes(df, 1)), :]
    else
        processed_df = df
    end

    n = nrow(processed_df)
    idx1 = floor(Int, pct1 * n)
    idx2 = floor(Int, pct2 * n)

    df1 = processed_df[1:idx1, :]
    df2 = processed_df[idx1+1:idx1+idx2,:]
    df3 = processed_df[idx1+idx2+1:end, :]

    return df1, df2, df3
end


# Call the split and shuffling function: 
training, testing, validation = split_df_percent(data, 0.6, 0.2)

first(training, 5)

Row,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
,PoisonCl…,CapShape,Surface,MColors,Bruised,Enum…,GillAtta…,Spacing,GillSize,String1,StalkSha…,Any,Surface,Surface,MColors,MColors,VeilType,MColors,Any,Enum…,MColors,Populati…,Habitat
1,Edible,Knobbed,Scaly,Brown,Bruises,None,Free,Close,Broad,w,Enlarging,Missing,Smooth,Smooth,White,White,Partial,White,2,Evanescent,White,Clustered,Waste
2,Edible,Flat,Fibrous,White,NoBruises,None,Free,Crowded,Broad,n,Tapering,Equal,Fibrous,Smooth,White,White,Partial,White,1,Evanescent,Brown,Scattered,Grasses
3,Poisonous,Convex,Scaly,Yellow,NoBruises,Foul,Free,Close,Broad,p,Enlarging,Bulbous,Silky,Silky,Buff,Pink,Partial,White,1,Large,Chocolate,Several,Paths
4,Edible,Flat,Scaly,Brown,Bruises,None,Free,Close,Broad,n,Tapering,Bulbous,Smooth,Smooth,Gray,Pink,Partial,White,1,Pendant,Brown,Several,Woods
5,Edible,Bell,Fibrous,White,NoBruises,None,Free,Crowded,Broad,p,Enlarging,Missing,Smooth,Smooth,White,White,Partial,White,2,Pendant,White,Numerous,Grasses


In [ ]:
# Define model
num_columns = ncol(training)

# Single layer with logistic regression
model = Dense(num_columns-1 => 1, sigmoid)

In [ ]:
# Setup model and RNG
dev = gpu_device()
rng = Random.default_rng()
opt = Adam(0.03f0)

vjp_rule = AutoZygote()
lossfn = MSELoss()

ps, st = Lux.setup(rng, model) |> dev

In [ ]:
# X and Y samples
x_samples = data[:, 2:end]
y_samples = data[:, 1]

first(x_samples, 5), y_samples

In [ ]:
# Training time
function train!(model, ps, st, opt, epochs)
    tstate = Training.TrainState(model, ps, st, opt)
    for epoch in 1:epochs
        grads, loss, _, tstate = Training.single_train_step!(vjp_rule, lossfn, (x_samples, y_samples), tstate)
    end
    return tstate.model, tstate.parameters, tstate.states
end

model, ps, st = train!(model, ps, st, opt, 100)